In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pypianoroll
from music21 import midi
from music21 import note, stream, duration
from music21 import converter
from MuseGAN_original import MuseGAN
from util_music import loaders
from tensorflow.keras.models import load_model
import tensorflow as tf

In [ ]:
from music21 import *
us = environment.UserSettings()
us['musescoreDirectPNGPath'] = '/usr/bin/mscore'
us['directoryScratch'] = '/tmp'

In [ ]:
# run params
SECTION = 'compose'
RUN_ID = '1000'
DATA_NAME = 'lpd_17' #'chorales'
# FILENAME = 'Jsb16thSeparated.npz'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

In [ ]:
RUN_FOLDER

## data

In [ ]:
BATCH_SIZE = 64
n_bars = 2
n_steps_per_bar = 16
n_pitches = 84
n_tracks = 8

#data_binary_2, data_ints_2, raw_data = loaders.load_music(DATA_NAME, FILENAME, n_bars, n_steps_per_bar)
#print(data_binary_2.shape)
#print(data_ints_2.shape)
# data_binary = np.squeeze(data_binary)

In [ ]:
data_binary = np.load('./run/dataset3.npy')
print(data_binary.shape)

In [ ]:
data_binary = data_binary.reshape(-1, n_bars, n_steps_per_bar, n_pitches, 17)
print(data_binary.shape)

In [ ]:
gan = MuseGAN(input_dim = data_binary.shape[1:-1] + (8,)
        , critic_learning_rate = 0.001
        , generator_learning_rate = 0.001
        , optimiser = 'adam'
        , grad_weight = 10
        , z_dim = 32
        , batch_size = BATCH_SIZE
        , n_tracks = n_tracks
        , n_bars = n_bars
        , n_steps_per_bar = n_steps_per_bar
        , n_pitches = n_pitches
        )

In [ ]:
gan.load_weights(RUN_FOLDER , None)

In [ ]:
gan.generator.summary()

In [ ]:
gan.critic.summary()

# view sample score

In [ ]:
chords_noise = np.random.normal(0, 1, (1, gan.z_dim))
style_noise = np.random.normal(0, 1, (1, gan.z_dim))
melody_noise = np.random.normal(0, 1, (1, gan.n_tracks, gan.z_dim))
groove_noise = np.random.normal(0, 1, (1, gan.n_tracks, gan.z_dim))

In [ ]:
def get_conditioned_track(midi=None, phrase_length=192, beat_resolution=12):
    
    if not isinstance(midi, str):
        # ----------- Generation from preprocessed dataset ------------------
        sample_x = midi
        sample_c = np.expand_dims(sample_x[..., 0], -1)
    else:
        # --------------- Generation from midi file -------------------------
        midi_file = midi

        parsed = pypianoroll.read(midi_file)
        parsed.set_resolution(beat_resolution)

        sample_c = parsed.tracks[0].pianoroll.astype(np.float32)
        
        # Remove initial steps that have no note-on events
        first_non_zero = np.nonzero(sample_c.sum(axis=1))[0][0]
        
        # Use the first 'phrase_length' steps as the primer
        sample_c = sample_c[first_non_zero: first_non_zero + phrase_length]

        # Binarize data (ignore velocity value)
        sample_c[sample_c > 0] = 1
        sample_c[sample_c <= 0] = -1

        sample_c = np.expand_dims(np.expand_dims(sample_c, 0), -1)  # 1, 32, 128, 1
        sample_c = sample_c[:, :, 24:24+84, :]
        sample_c = sample_c.reshape(-1, 2, 16, 84, 1)
    sample_c = tf.convert_to_tensor(sample_c, dtype=tf.float32)
    return sample_c

In [ ]:
y = get_conditioned_track(midi ='./input_midi/보고싶다.mid')# (2, 16, 84, 1)
print(y.shape)

In [ ]:
gen_scores = gan.generator.predict([chords_noise, style_noise, melody_noise, groove_noise, y], steps=1)

In [ ]:
np.argmax(gen_scores[0,0,0:4,:,3], axis = 1)

In [ ]:
filename = 'example'
gan.notes_to_midi(RUN_FOLDER, gen_scores, filename)
gen_score = converter.parse(os.path.join(RUN_FOLDER, 'samples/{}.midi'.format(filename)))
gen_score.show()

In [ ]:
gan.draw_score(gen_scores, 0)

# find the closest match

In [ ]:
def find_closest(data_binary, score):
    current_dist = 99999999
    current_i = -1
    for i, d in enumerate(data_binary):
        dist = np.sqrt(np.sum(pow((d - score),2)))
        if dist < current_dist:
            current_i = i
            current_dist = dist
        
    return current_i
    

In [ ]:
closest_idx = find_closest(data_binary[:, :, :, :, :8], gen_scores[0])
closest_data = data_binary[[closest_idx]]
print(closest_idx)

In [ ]:
closest_data.shape

In [ ]:
filename = 'closest'
gan.notes_to_midi(RUN_FOLDER, closest_data[:, :, :, :, :8],filename)
closest_score = converter.parse(os.path.join(RUN_FOLDER, 'samples/{}.midi'.format(filename)))
print('original')
gen_score.show()
print('closest')
closest_score.show()

# changing the chords noise

In [ ]:
chords_noise_2 = 5 * np.ones((1, gan.z_dim))

In [ ]:
chords_scores = gan.generator.predict([chords_noise_2, style_noise, melody_noise, groove_noise, y], steps=1)

In [ ]:
filename = 'changing_chords'
gan.notes_to_midi(RUN_FOLDER, chords_scores, filename)
chords_score = converter.parse(os.path.join(RUN_FOLDER, 'samples/{}.midi'.format(filename)))
print('original')
gen_score.show()
print('chords noise changed')
chords_score.show()

# changing the style noise

In [ ]:
style_noise_2 = 5 * np.ones((1, gan.z_dim))

In [ ]:
style_scores = gan.generator.predict([chords_noise, style_noise_2, melody_noise, groove_noise, y], steps=1)

In [ ]:
filename = 'changing_style'
gan.notes_to_midi(RUN_FOLDER, style_scores, filename)
style_score = converter.parse(os.path.join(RUN_FOLDER, 'samples/{}.midi'.format(filename)))
print('original')
gen_score.show()
print('style noise changed')
style_score.show()

# changing the melody noise

In [ ]:
melody_noise_2 = np.copy(melody_noise)
melody_noise_2[0,0,:] = 5 * np.ones(gan.z_dim)

In [ ]:
melody_scores = gan.generator.predict([chords_noise, style_noise, melody_noise_2, groove_noise, y], steps=1)

In [ ]:
filename = 'changing_melody'
gan.notes_to_midi(RUN_FOLDER, melody_scores, filename)
melody_score = converter.parse(os.path.join(RUN_FOLDER, 'samples/{}.midi'.format(filename)))
print('original')
gen_score.show()
print('melody noise changed')
melody_score.show()

# changing the groove noise

In [ ]:
groove_noise_2 = np.copy(groove_noise)
groove_noise_2[0,3,:] = 5 * np.ones(gan.z_dim)

In [ ]:
groove_scores = gan.generator.predict([chords_noise, style_noise, melody_noise, groove_noise_2, y], steps=1)

In [ ]:
filename = 'changing_groove'
gan.notes_to_midi(RUN_FOLDER, groove_scores, filename)
groove_score = converter.parse(os.path.join(RUN_FOLDER, 'samples/{}.midi'.format(filename)))
print('original')
gen_score.show()
print('groove noise changed')
groove_score.show()